In [5]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [6]:
url = "https://www.sahibinden.com/ilan/emlak-konut-satilik-urla-kalabak-kapanmayacak-manzarali-emsalsiz-mulk-808449106/detay"

In [ ]:
html = requests.get(url)
html = html.content

# get_sahibinden_record

In [41]:
def ConvertStringToList(string):
    li = list(string.split(" "))
    return li

def get_sahibinden_link(html):
    soup = BeautifulSoup(html, features="lxml") 
    url = soup.find('link', {'rel': 'alternate'})['href']
    return(url)

def get_sahibinden_record(html):
    record = {}
    soup = BeautifulSoup(html, features="lxml")
    soup = soup.find(id = "classifiedDetail")
    
    record["İlan"] = soup.find('h1').text
    
    fiyatBilgisi = soup.find('h3').text.strip()
    fiyatBilgisi = ConvertStringToList(fiyatBilgisi)
    
    M2fiyatBilgisi = soup.find('h4')
    
    detailKeys = soup.find('ul',class_='classifiedInfoList').find_all('strong')
    Keys = [detail.text.strip() for detail in detailKeys]
        
    detailValues = soup.find('ul',class_='classifiedInfoList').find_all('span')
    Values = [detail.text.strip() for detail in detailValues]
        
    record.update(dict(zip(Keys,Values)))
    record["Fiyat"] = fiyatBilgisi[0]
        
    if M2fiyatBilgisi == None:
        
        record["URL"] = get_sahibinden_link(html)
        
        record = pd.Series(record)
        record = record.to_frame()
        record = record.T
        
    else:
        M2fiyatBilgisi = soup.find('h4').text.strip()
        M2fiyatBilgisi = ConvertStringToList(M2fiyatBilgisi)
        
        record["M2 Fiyatı"] = M2fiyatBilgisi[2] + " " + M2fiyatBilgisi[3]
        record["URL"] = get_sahibinden_link(html)
        
        record = pd.Series(record)
        record = record.to_frame()
        record = record.T
        
    return(record)

In [42]:
get_sahibinden_record(html)

,İlan,Ad,Ad Date,Category,Type,Subtype,Brand,Year,Km,Vites,Capacity (Bed),From,Exchangeable,Condition,Fiyat,URL
0,ULTRA CARAVANS FARKI İLE TAM DONANIMLI MOTOKAR...,937344795,10 July 2021,"Campervan, Caravan & RV",Motorhomes,Panelvan (Camper),Peugeot,2015,120.000,Manuel,2,Store,Yes,Used,"428,000",https://www.sahibinden.com/ilan/vasita-karavan...


In [43]:
df = get_sahibinden_record(html)
df2 = get_sahibinden_record(html)

with pd.ExcelWriter('record.xlsx') as writer: 
    
    if "KM" or "Km" in get_sahibinden_record(html).keys():
        df.to_excel(writer, sheet_name='Araç') 
        
    elif "M2 Fiyatı" in get_sahibinden_record(html).keys():
        df2.to_excel(writer, sheet_name='Arazi') 
        
    else:
        df.to_excel(writer, sheet_name='Sheet') 
    